<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>



# Demo 8.3: Web Scraping

INSTRUCTIONS:

- Run the cells
- Observe and understand the results
- Answer the questions

# Web Scraping in Python (using BeautifulSoup)

# Basics HTML
Before starting with the code, let’s understand the basics of HTML and some rules of scraping.

## HTML tags
Below is the source code for a simple HTML webpage.

    <!DOCTYPE html>  
    <html>  
        <head>
        </head>
        <body>
            <h1> First Scraping </h1>
            <p> Hello World </p>
        <body>
    </html>
    
This is the basic syntax of an HTML webpage. Every `<tag>` serves a block inside the webpage:
1. `<!DOCTYPE html>` HTML documents must start with a type declaration.
2. The HTML document is contained between `<html>` and `</html>`.
3. The meta and script declaration of the HTML document is between `<head>` and `</head>`.
4. The visible part of the HTML document is between `<body>` and `</body>` tags.
5. Title headings are defined with the `<h1>` through `<h6>` tags.
6. Paragraphs are defined with the `<p>` tag.

Other useful tags include `<a>` for hyperlinks, `<table>` for tables, `<tr>` for table rows, and `<td>` for table columns.

Also, HTML tags sometimes come with `id` or `class` attributes. The `id` attribute specifies a unique id for an HTML tag and the value must be unique within the HTML document. The `class` attribute is used to define equal styles for HTML tags with the same class. We can make use of these ids and classes to help us locate the data we want.

## Scraping Rules
1. **Always** check a website’s **Terms and Conditions** before you scrape it. Be careful to read the statements about legal use of data. Usually, the retrieved data should not be used for commercial purposes.
2. **Do not** request data from the website too aggressively with a program (also known as spamming), as this may break the website. Make sure the program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.
3. The layout of a website may change from time to time, so make sure to revisit the site and rewrite the code as needed

## Inspecting the Page
Let’s take one page from the **Memory Alpha** website as an example.

To investigate some relationships let's get the links from this page.

Open the web page on [Prinadora](http://memory-alpha.wikia.com/wiki/Prinadora) with the browser and inspect it.

Hover the cursor on the text and follow the shaded box surrounding the main text.

From the result, check the main text inside a few levels of HTML tags, which is `<section id="WikiaPage" ...>` → `<div class="WikiaPageContentWrapper">` → `<article id="WikiaMainContent" class="WikiaMainContent">`.

The unique location of the data is now known.

In [ ]:
## Import Libraries
import regex as re

from urllib.parse import unquote
import urllib3
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

### Define the content to retrieve (webpage's URL)

In [ ]:
# specify the url
quote_page = 'http://memory-alpha.wikia.com/wiki/Prinadora'

### Retrieve the page
- Require Internet connection

In [ ]:
# query the website and return the html to the variable ‘page’
http = urllib3.PoolManager()
r = http.request('GET', quote_page)
if r.status == 200:
    page = r.data
    print('Type of the variable \'page\':', page.__class__.__name__)
    print('Page Retrieved. Request Status: %d, Page Size: %d' % (r.status, len(page)))
else:
    print('Some problem occurred. Request Status: %s' % r.status)

Type of the variable 'page': bytes
Page Retrieved. Request Status: 200, Page Size: 235389


### Convert the stream of bytes into a BeautifulSoup representation

In [ ]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page, 'html.parser')
print('Type of the variable \'soup\':', soup.__class__.__name__)

Type of the variable 'soup': BeautifulSoup


### Check the content
- The HTML source
- Includes all tags and scripts
- Can be long!

In [ ]:
print(soup.prettify()[:1000])

### Check the HTML's Title

In [ ]:
print('Title tag :%s:' % soup.title)
print('Title text:%s:' % soup.title.string)

### Wikimedia's article
- Wiki pages use the tag `article` for the actual content of the page

        <article class="WikiaMainContent" id="WikiaMainContent">
            <div class="WikiaMainContentContainer" id="WikiaMainContentContainer">
                <div class="WikiaArticle" id="WikiaArticle">

In [ ]:
tag = 'article'
article = soup.find_all(tag)[0]
print('Type of the variable \'article\':', article.__class__.__name__)

### Get some of the text
- Plain text without HTML tags

In [ ]:
# show the first 500 characters after removing redundant newlines
print(re.sub(r'\n\n+', '\n', article.text)[:500])

### Find the links in the text

In [ ]:
# identify the type of tag to retrieve
tag = 'a'
# create a list with the links from the `<a>` tag
tag_list = [t.get('href') for t in article.find_all(tag)]
tag_list

In [ ]:
# keep only the links to the wiki itself
tag_list = [t[6:] for t in tag_list if (t) and (t.startswith('/wiki/'))]
tag_list

In [ ]:
# create a filter for undesired links
filter  = '(%s)' % '|'.join([
    'Category:',
    'File:',
    'Help:',
    'Memory_Alpha:',
    'Portal:',
    'action=',
    'Special:',
    'Star_Trek:',
    'Star_Trek_',
    'Talk:'
])
# remove the links that are found in the filter
tag_list = [t for t in tag_list if not re.search(filter, t)]
tag_list

In [ ]:
# remove duplicates
tag_list = list(set(tag_list))
tag_list

In [ ]:
# convert escaped sequences
tag_list = [unquote(t) for t in tag_list]
tag_list

In [ ]:
# convert underscore to space
tag_list = [re.sub('_', ' ', t) for t in tag_list]
tag_list

In [ ]:
# order the list
tag_list.sort()
tag_list

### Create a filter for unwanted types of articles

In [ ]:
filter  = '(%s)' % '|'.join([
    'episode',
    'lternate_reality', # both Alternate_reality and alternate_reality
    'mirror',
    'rank',
    'production',
    'Season'
])
# remove the links that are found in the filter
tag_list = [t for t in tag_list if not re.search(filter, t)]
tag_list



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



